In [ ]:
from langchain.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate
from langchain_core.prompts import PromptTemplate
from langchain_ollama.llms import OllamaLLM

In [ ]:
llm = OllamaLLM(model="deepseek-r1:1.5b", reasoning=False)

# 1. Prompt template

In [ ]:
prompt_template = PromptTemplate(
    input_variables=["topic"], template="Tell me a joke about {topic}."
)

prompt = prompt_template.invoke({"topic": "cats"})
print(prompt)

# 2. Chat: From template

In [ ]:
# Example 1: Create a ChatPromptTemplate using a template string
prompt_template = ChatPromptTemplate.from_template("Tell me a joke about {topic}.")

print("----- Prompt -----")
prompt = prompt_template.invoke({"topic": "cats"})
print(prompt)

print("\n----- Response -----")
result = llm.invoke(prompt)
print(result)

# 3. Chat: From messsages

In [ ]:
# Example 2: Prompt with System and Human Messages (Using Tuples)
messages = [
    ("system", "You are a comedian who tells jokes about {topic}."),
    ("human", "Tell me {joke_count} jokes."),
]
prompt_template = ChatPromptTemplate.from_messages(messages)

print("----- Prompt -----")
prompt = prompt_template.invoke({"topic": "lawyers", "joke_count": 3})
print(prompt)

print("\n----- Response -----")
result = llm.invoke(prompt)
print(result)

# 4. Few Shot Prompting

In [ ]:
# Setup the template
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}"),
    ]
)

# We define a list of examples with dictionaries containing the correct input and output keys
examples = [
    {"input": "Here is query #1", "output": "Here is the answer #1"},
    {"input": "Here is query #2", "output": "Here is the answer #2"},
    {"input": "Here is query #3", "output": "Here is the answer #3"},
]

few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)
# Here is the formatted prompt
print(few_shot_prompt.format())

# 5. Chain of Thought Prompting

In [ ]:
# Define the chain-of-thought prompt template
cot_system_prompt = """
Be a helpful assistant and answer the user's question.

To answer the question, you must:

- List systematically and in precise detail all
  subproblems that need to be solved to answer the
  question.
- Solve each sub problem INDIVIDUALLY and in sequence.
- Finally, use everything you have worked through to
  provide the final answer.
"""

cot_prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", cot_system_prompt),
        ("user", "{query}"),
    ]
)

cot_pipeline = cot_prompt_template | llm

In [ ]:
query = "How many keystrokes are needed to type the numbers from 1 to 500?"

result = cot_pipeline.invoke({"query": query})
print(result)